In [1]:
import os
import shutil, subprocess, time
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
%matplotlib inline
from pydeseq2.dds import DeseqDataSet
from pydeseq2.dds import DefaultInference
from pydeseq2.ds import DeseqStats
from collections import Counter

/home/ivan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Verification sgRNA ABE Screen

In [2]:
os.chdir('/media/ivan/Extreme SSD/Verification ABE Screen')

In [3]:
#in terminal
gunzip *.gz


cutadapt -g TGGAAAGGACGAAACACC...GTTTTAGAGCTAGAAATAGCAAGTTA -j 10 -m 18 -M 23 I75A714_CKDL220019605-1A_HHT7GCCX2_L3_1.fq -o test_cut2.fq

bowtie-build Verification_sgRNA.fasta Veri
bowtie -n 2 -p 20 -q --best --norc -x BCR_ABL_Control_sgRNA test_cut2.fq > results.txt

SyntaxError: invalid decimal literal (2906377291.py, line 5)

In [27]:
for file in os.listdir():
    if "Veri_" in file:
        subprocess.call(['cutadapt', '-g', 'TGGAAAGGACGAAACACC...GTTTTAGAGCTAGAAAT', '-j', '20', '-m', '17', '-M', '23',
                     file , '-o', "Trimmed/" + file])

This is cutadapt 3.5 with Python 3.10.12
Command line parameters: -g TGGAAAGGACGAAACACC...GTTTTAGAGCTAGAAAT -j 20 -m 17 -M 23 Veri_L1713.fq -o Trimmed/Veri_L1713.fq
Processing reads on 20 cores in single-end mode ...
Finished in 0.22 s (3 µs/read; 19.27 M reads/minute).

=== Summary ===

Total reads processed:                  72,185
Reads with adapters:                    68,634 (95.1%)

== Read fate breakdown ==
Reads that were too short:                 143 (0.2%)
Reads that were too long:               16,041 (22.2%)
Reads written (passing filters):        56,001 (77.6%)

Total basepairs processed:    10,827,750 bp
Total written (filtered):      1,131,603 bp (10.5%)

=== Adapter 3 ===

Sequence: TGGAAAGGACGAAACACC...GTTTTAGAGCTAGAAAT; Type: linked; Length: 18+17; 5' trimmed: 68634 times; 3' trimmed: 68634 times

Minimum overlap: 3+3
No. of allowed errors:
1-9 bp: 0; 10-18 bp: 1

No. of allowed errors:
1-9 bp: 0; 10-17 bp: 1

Overview of removed sequences at 5' end
length	count	expe

In [4]:
os.chdir('/media/ivan/Extreme SSD/Verification ABE Screen/Trimmed')

bowtie-build Verification_sgRNA.fasta Veri

In [5]:
Veri_samples = ['Veri_L1713', 'Veri_L2622', 'Veri_L2713', 'Veri_L1622', 'Veri_plasmid']

In [6]:
for d in Veri_samples:
    subprocess.call(['bowtie', '-n', '2', '-p', '20', '-q', '--norc', '--best', '-l', '17','-x', 'Veri', d + '.fq', d + '.txt'])

# reads processed: 56001
# reads with at least one alignment: 54769 (97.80%)
# reads that failed to align: 1232 (2.20%)
Reported 54769 alignments
# reads processed: 197454
# reads with at least one alignment: 194822 (98.67%)
# reads that failed to align: 2632 (1.33%)
Reported 194822 alignments
# reads processed: 67676
# reads with at least one alignment: 61852 (91.39%)
# reads that failed to align: 5824 (8.61%)
Reported 61852 alignments
# reads processed: 197121
# reads with at least one alignment: 194255 (98.55%)
# reads that failed to align: 2866 (1.45%)
Reported 194255 alignments
# reads processed: 46932
# reads with at least one alignment: 34422 (73.34%)
# reads that failed to align: 12510 (26.66%)
Reported 34422 alignments


In [7]:
sgNames_fasta = open("Verification_sgRNA.fasta", "r").read().splitlines()[::2]
sgNames = [sg[1:] for sg in sgNames_fasta]
ScreenDF = pd.DataFrame(sgNames, columns=['sgName'])
ScreenDF.head()

,sgName
0,BCRABL_1776
1,BCRABL_1782
2,BCRABL_1792
3,BCRABL_1800
4,BCRABL_1808


In [8]:
for d in Veri_samples:
    Bowtie_output = open((d+".txt"), "r").read().split('\t')
    sgRNA_IDs = Bowtie_output[2::7] #sgRNA ID list
    sg_Frequency = Counter(sgRNA_IDs)
    sgCounts = []
    for sgN in sgNames:
        sgCounts.append(sg_Frequency[sgN])
    ScreenDF[d] = sgCounts
    print([d, len(sgRNA_IDs)])

['Veri_L1713', 54769]
['Veri_L2622', 194822]
['Veri_L2713', 61852]
['Veri_L1622', 194255]
['Veri_plasmid', 34422]


In [9]:
ScreenDF.to_csv('Veri_sgRNA_Screen_Counts.csv')

In [4]:
ScreenDF = pd.read_csv('Veri_sgRNA_Screen_Counts.csv').iloc[:, 1:]

In [10]:
Veri_samples = ['Veri_L1713', 'Veri_L2622', 'Veri_L2713', 'Veri_L1622', 'Veri_plasmid']

In [11]:
meta = pd.DataFrame(Veri_samples, columns=['sample'])
meta['condition'] = ['D19', 'D0', 'D19', 'D0', 'Plasmid']
meta = meta.set_index('sample')

In [12]:
meta

,condition
sample,
Veri_L1713,D19
Veri_L2622,D0
Veri_L2713,D19
Veri_L1622,D0
Veri_plasmid,Plasmid


In [13]:
ScreenDF2 = ScreenDF.set_index('sgName').T

In [14]:
inference = DefaultInference(n_cpus=20)
dds = DeseqDataSet(
    counts=ScreenDF2,
    metadata=meta,
    design_factors="condition",
    refit_cooks=True,
    n_cpus=20)
dds.deseq2()

Fitting size factors...
... done in 0.00 seconds.

/home/ivan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/ivan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/home/ivan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a s

In [15]:
DESeq2_Normed_Counts = pd.DataFrame(dds.layers['normed_counts'], index=ScreenDF2.index, columns=ScreenDF2.columns).T
DESeq2_Normed_Freq = DESeq2_Normed_Counts.div(DESeq2_Normed_Counts.sum(axis=0), axis=1)
DESeq2_Normed_Freq.to_csv('DESeq2_Normed_Freq_Veri_Screen.csv')

In [16]:
ABE_Veri = DeseqStats(dds, contrast=["condition", "D19", "D0"])
ABE_Veri.summary()

Running Wald tests...


Log2 fold change & Wald test p-value: condition D19 vs D0
                baseMean  log2FoldChange     lfcSE       stat        pvalue  \
sgName                                                                        
BCRABL_1776  1735.551038       -0.790591  0.321833  -2.456528  1.402870e-02   
BCRABL_1782  3983.107664        1.109251  0.096126  11.539535  8.337423e-31   
BCRABL_1792  3406.842440       -0.330971  0.168121  -1.968649  4.899343e-02   
BCRABL_1800  2052.655138       -0.291224  0.270226  -1.077705  2.811654e-01   
BCRABL_1808  7263.811038        1.142637  0.161960   7.055044  1.725457e-12   
BCRABL_1811  2682.252305       -0.729984  0.107051  -6.819001  9.167596e-12   
BCRABL_1817  1407.597094       -0.095338  0.097955  -0.973282  3.304132e-01   
BCRABL_1824  7211.442386        0.261897  0.069509   3.767811  1.646851e-04   
BCRABL_1829  3223.030997       -0.073291  0.246299  -0.297570  7.660313e-01   
BCRABL_1831  3333.278727       -0.775409  0.132757  -5.840812  5.194699e-

... done in 0.07 seconds.



In [17]:
ABE_Veri = ABE_Veri.results_df
ABE_Veri['Veri_growthrate'] = np.log(np.power(2, ABE_Veri['log2FoldChange'])*np.exp(0.055*19*24))/(19*24)

In [18]:
ABE_Veri['D0_Freq'] = DESeq2_Normed_Freq[['Veri_L1622', 'Veri_L2622']].mean(axis=1)

In [19]:
ABE_Veri.to_csv('ABE_Veri_DESEQ2_output.csv')

In [20]:
ABE_Veri[['D0_Freq', 'Veri_growthrate']].to_csv('ABE_Veri_Screen_Freq_GR240525.csv')

In [21]:
ABE_Veri_DESeq2 = ABE_Veri[['Veri_growthrate', 'D0_Freq','pvalue', 'padj']]
ABE_Veri_DESeq2.columns = [['Veri_growthrate', 'Veri_D0_Freq','Veri_pvalue', 'Veri_padj']]
ABE_Veri_DESeq2.reset_index().to_csv('ABE_Veri_DESeq2.csv')

In [22]:
ABE_Veri_DESeq2

,Veri_growthrate,Veri_D0_Freq,Veri_pvalue,Veri_padj
sgName,,,,
BCRABL_1776,0.053798,0.019475,1.402870e-02,2.338116e-02
BCRABL_1782,0.056686,0.035626,8.337423e-31,8.337423e-30
BCRABL_1792,0.054497,0.048364,4.899343e-02,7.537451e-02
BCRABL_1800,0.054557,0.025172,2.811654e-01,4.016649e-01
BCRABL_1808,0.056737,0.064944,1.725457e-12,6.901827e-12
BCRABL_1811,0.053890,0.044003,9.167596e-12,3.055865e-11
BCRABL_1817,0.054855,0.016548,3.304132e-01,4.240947e-01
BCRABL_1824,0.055398,0.087763,1.646851e-04,3.415306e-04
BCRABL_1829,0.054889,0.042443,7.660313e-01,7.660313e-01
